In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import pandas as pd
import math
import matplotlib.lines as mlines

In [2]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [3]:
def readNPDBFiles(path, n):
    file = open(path, "r")
    lines = file.readlines()
    pdbs = [(line.strip()[:4], line.strip()[4:5]) for line in lines[1:]]
    random.shuffle(pdbs)
    return pdbs[1:n]

In [4]:
def getPDBFile(pdb):
        try:
            url = "https://files.rcsb.org/view/{}.pdb".format(pdb)
            response = urllib.request.urlopen(url)
            data = response.read()
            text = data.decode('utf-8')
            text = text.split('\n')
            return(text)
        except:
            return([])

In [5]:
def getAtomCoordinates(lines, chain, aaNum, atom):
    atomLines = [line for line in lines if line.startswith("ATOM")]
    for line in atomLines:
        name = line[12:16].strip()
        chainID = line[21].strip()
        resSeq = line[22:26].strip()
        x = line[30:38].strip()
        y = line[38:46].strip()
        z = line[46:54].strip()
        if chainID == chain and resSeq == aaNum and atom == name:
            return(x, y, z)
    return None

def getDistance(lines, chain, aaNum1, atom1, aaNum2, atom2):
    atom1Coords = getAtomCoordinates(lines, chain, aaNum1, atom1)
    atom2Coords = getAtomCoordinates(lines, chain, aaNum2, atom2)
    
    if(atom1Coords is None or atom2Coords is None):
        return None
    
    x1 = float(atom1Coords[0])
    y1 = float(atom1Coords[1])
    z1 = float(atom1Coords[2])
    x2 = float(atom2Coords[0])
    y2 = float(atom2Coords[1])
    z2 = float(atom2Coords[2])
    
    dist = math.sqrt(math.pow((x2-x1), 2) + math.pow((y2-y1), 2) + math.pow((z2-z1), 2))
    return dist

def getBackboneAngle(lines, chain, aaNum):
    # CA of aaNum-1
    atom1Coords = getAtomCoordinates(lines, chain, aaNum, "N")
    atom1Coords = np.array(atom1Coords).astype(float)
    
    # C of aaNum-1
    atom2Coords = getAtomCoordinates(lines, chain, aaNum, "CA")
    atom2Coords = np.array(atom2Coords).astype(float)
    
    # N of aaNum
    atom3Coords = getAtomCoordinates(lines, chain, aaNum, "C")
    atom3Coords = np.array(atom3Coords).astype(float)
    
    #n-ca
    ba = atom1Coords - atom2Coords
 
    #c-ca
    bc = atom3Coords - atom2Coords
 
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)


In [6]:
def AnalyzePDB_Q7(pdbs):
    NtoCADistances = []
    CAtoCDistances = []
    backboneAngles = []
    processed = 1
    for pdb in pdbs:
        update_progress(processed / len(pdbs))
        print("Processing {}/{}".format(processed, len(pdbs)))
        lines = getPDBFile(pdb[0])
        aaNums = set()
        atoms = [line for line in lines if line.startswith("ATOM")]
        prevaaNum = -1
        for atom in atoms:
            chainID = atom[21].strip()
            aaNum = atom[22:26].strip()
            residueName = atom[17:20].strip()
            name = atom[12:16].strip()
            if(chainID == pdb[1] and prevaaNum != aaNum):
                NtoCADistance = getDistance(lines, chainID, aaNum, "N", aaNum, "CA")
                CAtoCDistance = getDistance(lines, chainID, aaNum, "CA", aaNum, "C")
                backboneAngle = getBackboneAngle(lines, chainID, aaNum)
                if(NtoCADistance is None or CAtoCDistance is None or backboneAngle is None):
                    continue
                NtoCADistances.append(NtoCADistance)
                CAtoCDistances.append(CAtoCDistance)
                backboneAngles.append(backboneAngle)
            prevaaNum = aaNum
        processed+=1
    return(np.array(NtoCADistances), np.array(CAtoCDistances), np.array(backboneAngles))

In [ ]:
%%time
pdbs = readNPDBFiles("cullpdb_pc30_res3.0_R1.0_d191017_chains18877", 18877)
NtoCADistances, CAtoCDistances, backboneAngles = AnalyzePDB_Q7(pdbs)

Progress: [#######-------------] 35.7%
Processing 6736/18876


In [ ]:
print("Nitrogen to Carbon Alpha distance mean is {}, std is {}.".format(NtoCADistances.mean(), NtoCADistances.std()))
print("Carbon Alpha to Carbon distance mean is {}, std is {}.".format(CAtoCDistances.mean(), CAtoCDistances.std()))
print("Angle of Nitrogen, Carbon Alpha, and Carbon mean is {}, std is {}.".format(backboneAngles.mean(), backboneAngles.std()))
      